**Import Librarys**

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re
from tensorflow.keras import Sequential
from tensorflow.keras import datasets,models,layers
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

2024-05-17 12:34:14.946764: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 12:34:14.946886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 12:34:15.223155: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('wordnet', "/kaggle/working/nltk_data/")
nltk.download('omw-1.4', "/kaggle/working/nltk_data/")
! unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora
! unzip /kaggle/working/nltk_data/corpora/omw-1.4.zip -d /kaggle/working/nltk_data/corpora
nltk.data.path.append("/kaggle/working/nltk_data/")

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data/...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data/...
Archive:  /kaggle/working/nltk_data/corpora/wordnet.zip
   creating: /kaggle/working/nltk_data/corpora/wordnet/
  inflating: /kaggle/working/nltk_data/corpora/wordnet/lexnames  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adv  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/nltk_data/c

**Load dataset**

In [ ]:
columns  = ["sentiment", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "latin-1"
dataset = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv',
                      encoding=DATASET_ENCODING , names=columns)

In [ ]:
dataset.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
dataset['sentiment'] = dataset['sentiment'].replace(4,1)

dataset = dataset[['sentiment','text']]

dataset.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
dataset4 = dataset[dataset['sentiment']==1]

dataset4.head()

,sentiment,text
800000,1,I LOVE @Health4UandPets u guys r the best!!
800001,1,im meeting up with one of my besties tonight! ...
800002,1,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800003,1,Being sick can be really cheap when it hurts t...
800004,1,@LovesBrooklyn2 he has that effect on everyone


**Text Preprocess**

In [ ]:
def clean_text(text):
    # remove all non-word characters
    text = re.sub(r'\W', ' ', text)
    # remove all digits
    text = re.sub(r'\d+', ' ', text)
    # remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # remove extra space
    text = re.sub(r'\s+',' ',text, flags=re.I)
    # remove punctuation
    text = re.sub(r'[^\w\s]',' ',text, flags=re.I)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [word for word in text if not word in stopwords]
    text = " ".join(text)
    return text

In [ ]:
dataset['text'] = dataset['text'].map(clean_text)

In [ ]:
dataset.head()

,sentiment,text
0,0,switchfoot http twitpic com zl awww bummer sh...
1,0,upset update facebook texting might cry result...
2,0,kenichan dived many time ball managed save re...
3,0,whole body feel itchy like fire
4,0,nationwideclass behaving mad see


**Spliting data**

In [ ]:
x,y = dataset['text'],dataset['sentiment']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,shuffle=True)

print("Train Data size:", len(x_train))
print("Test Data size", len(x_test))



Train Data size: 1280000
Test Data size 320000


**Tokenizer**

In [ ]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(x_train)  # Ensure consistent preprocessing

# Convert text to sequences of integers
X_train_sequences = tokenizer.texts_to_sequences(x_train)
X_test_sequences = tokenizer.texts_to_sequences(x_test)
max_sequence_length = 40

# padding
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)
pd.DataFrame({'text':tokenizer.sequences_to_texts(list(X_train_sequences)),'sequances':X_train_sequences})



,text,sequances
0,cathleenritt late dinner house,"[177204, 213, 262, 136]"
1,sasabelle nice one know else fails want groupi...,"[108532, 67, 17, 19, 365, 3688, 24, 8959, 4103..."
2,listening therealsavannah radio disney,"[259, 7231, 732, 1291]"
3,definitely least next monday morning,"[494, 243, 88, 217, 40]"
4,thejosh night,"[79503, 26]"
...,...,...
1279995,shower shower buna place afterwards,"[441, 441, 66799, 270, 2932]"
1279996,work,[10]
1279997,mother day shopping wow big thanks sister comi...,"[326, 2, 364, 166, 165, 33, 375, 184, 58, 63, ..."
1279998,mar sha time lazy figure,"[3632, 9204, 13, 723, 690]"


In [ ]:
print(f"After padding: {X_train_padded.shape}")
print(f"After padding:{X_test_padded.shape}")

After padding: (1280000, 40)
After padding:(320000, 40)


**First Model**

In [ ]:
vocabulary_size = len(tokenizer.word_index)+1
embedding_dim = 100

cnn=models.Sequential([
    layers.Embedding(vocabulary_size, embedding_dim,trainable=False),
    layers.Conv1D(filters=128,kernel_size=3,activation='relu'),
    layers.MaxPooling1D(2,2),
    layers.Conv1D(filters=64,kernel_size=3,activation='relu'),
    layers.MaxPooling1D(2,2),
    layers.Flatten(),
    layers.Dense(10,activation="relu"),
    layers.Dense(1, activation='sigmoid')
])


cnn.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])


In [ ]:
history = cnn.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test), batch_size=1024)


Epoch 1/10
  24/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5011 - loss: 0.6932 

I0000 00:00:1715949560.482999     113 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1715949560.500706     113 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.6034 - loss: 0.6470 - val_accuracy: 0.6708 - val_loss: 0.5925
Epoch 2/10
   9/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6863 - loss: 0.5848    

W0000 00:00:1715949571.748700     112 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.6814 - loss: 0.5841 - val_accuracy: 0.6812 - val_loss: 0.5829
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.6907 - loss: 0.5730 - val_accuracy: 0.6792 - val_loss: 0.5829
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.6968 - loss: 0.5658 - val_accuracy: 0.6866 - val_loss: 0.5768
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.7012 - loss: 0.5604 - val_accuracy: 0.6879 - val_loss: 0.5761
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.7048 - loss: 0.5559 - val_accuracy: 0.6889 - val_loss: 0.5756
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.7075 - loss: 0.5528 - val_accuracy: 0.6885 - val_loss: 0.5754
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.7095 - loss: 0.5502 - val_accuracy: 0.6864 - val_loss: 0.5787
Epoch 9/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7104 - loss: 0.5490 - val_accurac

In [ ]:
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (1024, 40, 100)        │    50,130,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (1024, 38, 128)        │        38,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (1024, 19, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (1024, 17, 64)         │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (1024, 8, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (1024, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1024, 10)             │         5,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (1024, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,335,129 (192.01 MB)

 Trainable params: 68,309 (266.83 KB)

 Non-trainable params: 50,130,200 (191.23 MB)

 Optimizer params: 136,620 (533.68 KB)

**Second Model**

In [ ]:
vocabulary_size = len(tokenizer.word_index) + 1
embedding_dim = 100

# Define CNN model
model = Sequential([
    layers.Embedding(vocabulary_size, embedding_dim, trainable=False),
    layers.Conv1D(filters=128, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
history = model.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test), batch_size=128)

Epoch 1/10
   44/10000 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - accuracy: 0.4866 - loss: 0.6942  

W0000 00:00:1715949691.853797     111 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


 9987/10000 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6726 - loss: 0.5845

W0000 00:00:1715949716.591662     111 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


10000/10000 ━━━━━━━━━━━━━━━━━━━━ 37s 3ms/step - accuracy: 0.6727 - loss: 0.5845 - val_accuracy: 0.7419 - val_loss: 0.5119
Epoch 2/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.7495 - loss: 0.5047 - val_accuracy: 0.7495 - val_loss: 0.5017
Epoch 3/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.7632 - loss: 0.4867 - val_accuracy: 0.7536 - val_loss: 0.4973
Epoch 4/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.7704 - loss: 0.4766 - val_accuracy: 0.7554 - val_loss: 0.4954
Epoch 5/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.7764 - loss: 0.4673 - val_accuracy: 0.7560 - val_loss: 0.4976
Epoch 6/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.7800 - loss: 0.4611 - val_accuracy: 0.7563 - val_loss: 0.4947
Epoch 7/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.7824 - loss: 0.4574 - val_accuracy: 0.7565 - val_loss: 0.4971
Epoch 8/10
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.7852 - lo

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (128, 40, 100)         │    50,130,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (128, 36, 128)         │        64,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (128, 18, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (128, 16, 64)          │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (128, 8, 64)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (128, 512)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (128, 256)             │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (128, 256)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (128, 128)             │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (128, 128)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (128, 1)               │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,889,565 (194.13 MB)

 Trainable params: 253,121 (988.75 KB)

 Non-trainable params: 50,130,200 (191.23 MB)

 Optimizer params: 506,244 (1.93 MB)

**Predict Sentiment**

In [ ]:
def predict_sentiment(model, text):
    preprocessed_text = clean_text(text)
    sequence = tokenizer.texts_to_sequences([preprocessed_text])
    padded_sequence = pad_sequences(sequence, maxlen=40)
    prediction = model.predict(padded_sequence)[0][0]
    if prediction >= 0.5:
        return 1
    else:
        return 0

In [ ]:
from sklearn.metrics import accuracy_score

# Test the first 10 sentences using 'cnn'
cnn_predictions = [predict_sentiment(cnn, text) for text in x_test[:10]]
cnn_accuracy = accuracy_score(y_test[:10], cnn_predictions)

# Test the first 10 sentences using 'model'
model_predictions = [predict_sentiment(model, text) for text in x_test[:10]]
model_accuracy = accuracy_score(y_test[:10], model_predictions)


print(f"First Model Accuracy: {cnn_accuracy * 100:.2f}%")
print(f"Sencond Model Accuracy: {model_accuracy * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
First Model Accuracy: 80.00%
Sencond Model Accuracy: 90.00%


In [ ]:
print("First Model",cnn_predictions)

First Model [0, 1, 0, 0, 0, 1, 1, 1, 1, 0]


In [ ]:
print("Second Model",model_predictions)

Second Model [0, 1, 0, 1, 0, 1, 1, 1, 1, 0]


In [ ]:
print("True Labels",np.array(y_test[:10]))

True Labels [1 1 0 1 0 1 1 1 1 0]
